<a href="https://colab.research.google.com/github/nshoo/Neural-Synthesis-Visualization/blob/main/pima_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

2022-08-12 17:52:59.537550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 17:52:59.537787: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# load data
data = pd.read_csv( 'diabetes.csv', names=[ 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome' ] )

for col in data:
    data[ col ] = data[ col ].astype( float )

rows = len(data)
print(rows)

print(data.shape[1])
data.head()
type(data["Pregnancies"])
data["Pregnancies"].shape
newData = data[["Pregnancies", "Age"]]
newData.head()


768
9


,Pregnancies,Age
0,6.0,50.0
1,1.0,31.0
2,8.0,32.0
3,1.0,21.0
4,0.0,33.0


In [3]:
# impute
for col in [ 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI' ]:
    mean = data[ data[ col ] != 0.0 ][ col ].mean()
    data[ col ] = data[ col ].replace( 0.0, mean )
    

In [4]:
# shuffle data "in place": https://stackoverflow.com/a/34879805
data = data.sample( frac = 1 ).reset_index( drop = True )

xs = data.copy()
ys = xs.pop( 'Outcome' ).to_frame( name = 'Outcome' )

test_len = len( xs ) // 5

train_xs, test_xs = xs.head( len( xs ) - test_len ), xs.tail( test_len )
train_ys, test_ys = ys.head( len( xs ) - test_len ), ys.tail( test_len )

norm = keras.layers.Normalization()
norm.adapt( train_xs )

model = keras.Sequential()
model.add( norm )
model.add( keras.layers.Dense( 9, input_dim=9, kernel_initializer='normal', activation='relu' ) )
model.add( keras.layers.Dense( 20, activation='relu' ) )
model.add( keras.layers.Dense( 20, activation='relu' ) )
model.add( keras.layers.Dense( 1, activation='relu' ) )
model.add( keras.layers.ReLU( max_value = 1.0 ) ) # for clamping
model.summary()

model.compile( loss = tf.losses.MeanSquaredError(), optimizer = tf.optimizers.Adam(), metrics=[ tf.keras.metrics.MeanSquaredError() ] )

model.fit( train_xs, train_ys, epochs = 10 )
model.evaluate( test_xs, test_ys )


2022-08-12 17:53:07.837317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-12 17:53:07.837521: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-12 17:53:07.837775: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kbjorner): /proc/driver/nvidia/version does not exist
2022-08-12 17:53:07.840184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 9)                 81        
                                                                 
 dense_1 (Dense)             (None, 20)                200       
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
 re_lu (ReLU)                (None, 1)                 0         
                                                        

[0.18605075776576996, 0.18605075776576996]

In [5]:
test_xs

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
615,7.0,83.0,78.0,26.00000,71.000000,29.3,0.767,36.0
616,1.0,117.0,88.0,24.00000,145.000000,34.5,0.403,40.0
617,3.0,82.0,70.0,29.15342,155.548223,21.1,0.389,25.0
618,3.0,129.0,92.0,49.00000,155.000000,36.4,0.968,32.0
619,0.0,128.0,68.0,19.00000,180.000000,30.5,1.391,25.0
...,...,...,...,...,...,...,...,...
763,1.0,95.0,60.0,18.00000,58.000000,23.9,0.260,22.0
764,6.0,111.0,64.0,39.00000,155.548223,34.2,0.260,24.0
765,1.0,95.0,74.0,21.00000,73.000000,25.9,0.673,36.0
766,9.0,106.0,52.0,29.15342,155.548223,31.2,0.380,42.0


In [6]:

test_data = pd.concat([test_xs, test_ys], axis = 1)
test_data
# below equal to test_xs
newData = test_data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]

f = open("pima_constraints.txt", "w")

n = 100
oneCounter = 0
zeroCounter = 0
for i in range(len(test_xs)):
  point = test_xs.values[i]
  outcome = test_ys.values[i]
  f.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), point)), 'true' if outcome == 1 else 'false'))
  if outcome == 1:
    oneCounter += 1
  else:
    zeroCounter += 1

f.close()

print(f"ones: {oneCounter}; zeros: {zeroCounter}")

ones: 50; zeros: 103


In [7]:
# counterfactual generation
x1 = pd.DataFrame( data = [ [ 3.0, 129.0, 92.0, 49.0, 155.0, 36.4, 0.968, 32.0 ] ] )
y1 = model.predict( x1 )
print( x1 )
print( y1 )
print()

x2 = pd.DataFrame( data = [ [ 3.0, 129.0, 92.0, 49.0, 155.0, 35, 0.968, 32.0 ] ] )
y2 = model.predict( x2 )
print( x2 )
print( y2 )


model.save( './pima_model' )

1/1 [==============================] - 0s 104ms/step
     0      1     2     3      4     5      6     7
0  3.0  129.0  92.0  49.0  155.0  36.4  0.968  32.0
[[0.5203932]]

1/1 [==============================] - 0s 21ms/step
     0      1     2     3      4   5      6     7
0  3.0  129.0  92.0  49.0  155.0  35  0.968  32.0
[[0.4962201]]
INFO:tensorflow:Assets written to: ./pima_model/assets
